In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from geopy.geocoders import Nominatim
from time import sleep

In [2]:
geolocator = Nominatim(user_agent="MozillaFirefox", timeout=10)

In [3]:
df = pd.read_csv('../../Data/03Limpio/03desastreslimpio.csv')

In [4]:
filter = df['Disaster Type'] == 'Earthquake'

df_earthquake = df[filter]

df_earthquake.head()

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,...,Longitude,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,Total Affected,Total Damages Adj
2,1902-0012-GTM,1902,12,Geophysical,Earthquake,Ground movement,Guatemala,GTM,Central America,Americas,...,-91,1902,4,18,1902,4,18,2000,0,843726
9,1905-0003-IND,1905,3,Geophysical,Earthquake,Ground movement,India,IND,Southern Asia,Asia,...,76.16,1905,4,4,1905,4,4,20000,0,812477
10,1906-0014-CHL,1906,14,Geophysical,Earthquake,Ground movement,Chile,CHL,South America,Americas,...,71.4,1906,8,16,1906,8,16,20000,0,3249907
11,1906-0002-COL,1906,2,Geophysical,Earthquake,Ground movement,Colombia,COL,South America,Americas,...,-78.46,1906,1,31,1906,1,31,400,0,0
15,1907-0006-CHN,1907,6,Geophysical,Earthquake,Ground movement,China,CHN,Eastern Asia,Asia,...,69.9,1907,10,21,1907,10,21,12000,0,0


In [5]:
df_earthquake.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1597 entries, 2 to 16617
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Dis No             1597 non-null   object
 1   Year               1597 non-null   int64 
 2   Seq                1597 non-null   int64 
 3   Disaster Subgroup  1597 non-null   object
 4   Disaster Type      1597 non-null   object
 5   Disaster Subtype   1597 non-null   object
 6   Country            1597 non-null   object
 7   ISO                1597 non-null   object
 8   Region             1597 non-null   object
 9   Continent          1597 non-null   object
 10  Location           1557 non-null   object
 11  Origin             21 non-null     object
 12  Associated Dis     342 non-null    object
 13  Dis Mag Value      1597 non-null   int64 
 14  Dis Mag Scale      1597 non-null   object
 15  Latitude           1547 non-null   object
 16  Longitude          1547 non-null   object

In [6]:
def limpiar_valor(valor):
    if pd.isna(valor):
        return valor
    elif 'N' in valor or 'E' in valor:
        return valor.replace('N', '').replace('E', '')
    elif 'S' in valor or 'W' in valor:
        return '-' + valor.replace('S', '').replace('W', '')
    else:
        return valor
    
df_earthquake['Latitude'] = df_earthquake['Latitude'].apply(limpiar_valor)
df_earthquake['Longitude'] = df_earthquake['Longitude'].apply(limpiar_valor)

C:\Users\blanc\AppData\Local\Temp\ipykernel_10512\1236789326.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_earthquake['Latitude'] = df_earthquake['Latitude'].apply(limpiar_valor)
C:\Users\blanc\AppData\Local\Temp\ipykernel_10512\1236789326.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_earthquake['Longitude'] = df_earthquake['Longitude'].apply(limpiar_valor)


In [7]:
df_earthquake[['Latitude', 'Longitude', 'Country']][700:800:]

,Latitude,Longitude,Country
5766,23.083,80.041,India
5779,37.661,57.291,Iran (Islamic Republic of)
5780,38.075,48.050,Iran (Islamic Republic of)
5781,33.938,-71.872,Iran (Islamic Republic of)
5786,43.0,13.0,Italy
...,...,...,...
7209,40.709,30.045,Turkey
7210,NaN,NaN,Turkey
7211,40.711,29.949,Turkey
7212,40.758,31.161,Turkey


In [8]:
for i, row in df_earthquake.iterrows():
    if pd.isnull(row['Location']):
        location = geolocator.geocode(row['Country'])
    else:
        location = geolocator.geocode(row['Location'])
    if location:
        df_earthquake.loc[i, 'Latitude'] = location.latitude
        df_earthquake.loc[i, 'Longitude'] = location.longitude
        sleep(2)

df_earthquake[['Latitude', 'Longitude']].isnull().sum()

GeocoderServiceError: Non-successful status code 502

In [30]:
def fill_lat_lon(row):
    if pd.isnull(row['Latitude']) and pd.isnull(row['Longitude']):
        location = geolocator.geocode(row['Country'])
        if location:
            row['Latitude'] = location.latitude
            row['Longitude'] = location.longitude
            sleep(1)
    return row


df_earthquake_fillnan = df_earthquake.apply(fill_lat_lon, axis=1)
df_earthquake_fillnan.isnull().sum()

Dis No                  0
Year                    0
Seq                     0
Disaster Subgroup       0
Disaster Type           0
Disaster Subtype        0
Country                 0
ISO                     0
Region                  0
Continent               0
Location               40
Origin               1576
Associated Dis       1255
Dis Mag Value          89
Dis Mag Scale           0
Latitude                0
Longitude               0
Start Year              0
Start Month             2
Start Day               3
End Year                0
End Month               2
End Day                 3
Total Deaths          399
Total Affected        277
Total Damages Adj     980
dtype: int64

In [31]:
def fill_anomalos_lat_lon(row):
    if abs(row['Latitude'])>90 and abs(row['Longitude'])>180:
    
        location = geolocator.geocode(row['Country'])
        if location:
            row['Latitude'] = location.latitude
            row['Longitude'] = location.longitude
            sleep(1)
    return row

df_earthquake_fillanomalos = df_earthquake_fillnan.apply(fill_anomalos_lat_lon, axis=1)
df_earthquake_fillanomalos.head()

TypeError: bad operand type for abs(): 'str'

In [28]:
print(df_earthquake['Latitude'].unique())
print(df_earthquake['Longitude'].unique())

['14' '32.04' '33.05' ... '38.424' '-4.326' '36.159']
['-91' '76.16' '71.4' ... '143.159' '36.034' '69.881']
